In [ ]:
import psycopg2
import matplotlib.pyplot as plt


connection = psycopg2.connect(
    database="piscineds",
    user="pbureera",
    password="mysecretpassword",
    host="localhost",
    port="5432",
)
cursor = connection.cursor()


In [ ]:
cursor.execute(
    """
    SELECT
        COUNT(product_id),
        AVG(price) AS mean,
        SQRT(VARIANCE(price)) AS std
        MIN(price) AS min,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY price) AS '25%',
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY price) AS '50%',
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY price) AS '75%',
        MAX(price) AS max,
    FROM customers
    WHERE event_type = 'purchase';
    """
)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stats = [(col, val) for col, val in zip(column_names, data[0])]
print("\n".join([f"{col}: {val}" for col, val in stats]))


In [ ]:
cursor.execute(
    """
    SELECT price FROM customers
    WHERE event_type = 'purchase'
    """
)
data = cursor.fetchall()
prices = [item[0] for item in data]
plt.boxplot(prices, vert=False)
plt.ylim(0.9, 1.1)
plt.xlabel("price")

plt.gca().set_facecolor('#EAEAF2')
plt.grid(visible=True, axis='x', color='white', linestyle='-', linewidth=0.7)
plt.show()

In [ ]:
plt.boxplot(prices, sym="", vert=False, patch_artist=True)
plt.ylim(0.9, 1.1)
plt.xlabel("price")

plt.gca().set_facecolor('#EAEAF2')
plt.grid(visible=True, axis='x', color='white', linestyle='-', linewidth=0.7)
plt.show()


In [ ]:
cursor.execute(
    """
    SELECT SUM(price) FROM customers
    WHERE event_type = 'cart' GROUP BY user_id
    """
)
data = cursor.fetchall()
prices = [item[0] for item in data]
plt.boxplot(prices, vert=False)
plt.ylim(0.9, 1.1)
plt.xlim(-10, 200)

plt.gca().set_facecolor('#EAEAF2')
plt.grid(visible=True, axis='x', color='white', linestyle='-', linewidth=0.7)
plt.show()

cursor.close()
connection.close()